Notebook to test and experiment with IG Classes
There are some trials to check HikerAPI, but there are no test of the post and comment treatment using Instagrapy

### Variable setting and other initial imports

In [ ]:
"""
main.py:
Example usage of the Instagram client, post manager, and music features to automate content posting.
"""

import os, pprint
import pandas as pd
from ig_auth import authenticate_and_save_session
from ig_client import IgClient
from ig_post_manager import IgPostManager


### 1. Authentication (If you don't have a session file yet)

In [ ]:
# 1. Authentication (Always attempt authentication first)
session_file = "cl_ig.pkl"
if not os.path.exists(session_file):  # Check if session file exists
    authenticate_and_save_session(session_file)

# Now, you should have a valid session, so create the client
insta_client = IgClient(session_file)
post_manager = IgPostManager(insta_client)


#### Initialize a list to store IgPost objects

In [ ]:
ig_posts = []
df_post = pd.DataFrame() 

### Test the basic APIs for p-mgmt

#### # 2. Post a Photo:

In [ ]:
# 2. Post a Photo:
photo_path = r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\IMG-20160212-WA0018.jpeg"
photo_caption = """Chasing sunsets along the enchanting Malecon in Miraflores, Lima. 🌅 
Embracing the golden hues as the sun dips below the horizon, 
painting the sky in a masterpiece of colors. 
Where's your favorite sunset spot? Share below! 

#SunsetMagic #MirafloresSunset 
#LimaSunsets #MaleconViews #GoldenHourGram #PeruvianSunsets #TravelDiaries #SunsetChasers #NaturePhotography #TravelGram #WanderlustAdventures"""

location_pk = 61047499  # Malecón Cisneros - Miraflores
ig_post = post_manager.upload_photo(photo_path, photo_caption, location_pk=location_pk)
ig_posts.append(ig_post) 


In [ ]:
# Create the Pandas DataFrame to save the above output
from ig_data import create_post_dataframe, save_post_dataframe

df_posts = create_post_dataframe(ig_posts)
print(df_posts)

# Save the dataframe
save_post_dataframe(df_posts, filename="my_instagram_posts_2.csv")

#### # 3. Post a Reel:

In [ ]:

# 3. Post a Reel:
video_path = r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\VID_20151108_161138142.mp4"  # Replace with your actual video path
video_caption = """Witness Otto, our four-legged skateboarding star, 
getting ready to breaking records with style and skill! 🐾🛹
#OttoTheSkateboardingDog 
#WorldRecordHolder #TalentOnFourPaws #DogSkateboarder #RecordBreakingPup #lima #peru 
# malecomcisneros #travelexperiences #funnyfacts @ottoskater"""

ig_post = post_manager.upload_reel(video_path, video_caption)

ig_posts.append(ig_post)  # Store the post object

In [ ]:
# Create the Pandas DataFrame to save the above output
from ig_data import create_post_dataframe, save_post_dataframe

df_posts = create_post_dataframe(ig_posts)
print(df_posts)

#### 4. Post a Carousel (Album) of Photos and Videos:

#### it never worked due to 
---------------------------------------------------------------------------
ResponseError                             Traceback (most recent call last)
ResponseError: too many 500 error responses

Possible Reasons:

Instagram Server Issues: Instagram's servers might be experiencing temporary problems or undergoing maintenance.
API Changes: Instagram might have made changes to their API that are causing unexpected behavior with Instagrapi.
Rate Limiting: If you've been uploading a lot of content in a short period, Instagram might be temporarily rate-limiting your account to prevent abuse.
Account Flagged: In rare cases, if your account has been flagged for suspicious activity, it could lead to errors when interacting with the API.

In [ ]:

# 4. Post a Carousel (Album) of Photos and Videos:
location_pk = 61047499  # Malecón Cisneros - Miraflores

paths = [
    r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\IMG_20151108_163642941.jpg",
    # r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\IMG_20151108_163629670.jpg",
    # r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\IMG-20151108-WA0001.jpg",
    r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\VID_20151108_162632485.mp4"
]

album_caption = "🐾 Throwback to November 2015 on the Malecón in Miraflores, Lima, where Otto the skateboarding bulldog made history! 🛹 ",
"🐶 Otto cruised his way into the Guinness World Records by skating through the longest human tunnel ever attempted by a dog.", 
"🏅A paw-some achievement for this furry athlete!   ", 
"#SkateboardingDog #GuinnessWorldRecord #Miraflores #Lima #Peru #TravelPeru #DogsofInstagram #AdventureDog #FourLeggedFun"

ig_post = post_manager.upload_album(paths, album_caption)

ig_posts.append(ig_post)  # Store the post object

df_posts = create_post_dataframe(ig_posts)
print(df_posts)

### save info and printing  --- part is done on the above cells

In [ ]:
# Create the Pandas DataFrame
df_posts = create_post_dataframe(ig_posts)
print(df_posts)

# Save the dataframe
save_post_dataframe(df_posts, filename="my_instagram_posts.csv")

In [ ]:

# 5. Upload a Reel with Music:
reel_video_path = "path/to/your/reel.mp4"
reel_caption = "Having fun with this new trend! #dance #music"


### It seems the following advanced methods (# 6 and #7)
#### for incorporating background music
#### are not available in the API Library - check the source API documentation


Type of music_results: It's likely a list or some iterable collection of objects representing music tracks found in the search.
Arguments/Parameters: Each music track object in music_results probably has attributes like:
id: A unique identifier for the music track.
title: The title of the song.
artist: The name of the artist.
album: The name of the album (if applicable).
cover_url: A URL to the album cover image.


In [ ]:
print(dir(post_manager))

In [ ]:

# Option A: Search for music on Instagram


search_query = "Manuel Carrasco"
music_results = post_manager.search_music(search_query)

if music_results:  # Check if any results were found
    for track in music_results:  # Iterate through the list of music tracks
        print("----------------------")
        print(f"ID: {track.id}")
        print(f"Title: {track.title}")
        print(f"Artist: {track.artist}")
        # Add more print statements for other attributes you want to display (e.g., album, cover_url, audio_url)

else:
    print("No music tracks found for the given search query.")

### Test the more advance p-mgmt APIs

In [ ]:
#### This Method is not working - but needs to be reviewed through the Instagrapi community


video_caption = "🐾 Throwback to November 2015 on the Malecón in Miraflores, Lima, where Otto the skateboarding bulldog made history! 🛹 ",
"🐶 Otto cruised his way into the Guinness World Records by skating through the longest human tunnel ever attempted by a dog.", 
"🏅A paw-some achievement for this furry athlete!   ", 
"#SkateboardingDog #GuinnessWorldRecord #Miraflores #Lima #Peru #TravelPeru #DogsofInstagram #AdventureDog #FourLeggedFun"


# Upload video with a music track
video_path =  r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\VID_20151108_163255451.mp4"
music_path =  r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\music_fx_emotion_driven_expectation__with_classic_mu (1).mp3"
post = post_manager.upload_video_with_music(video_path, music_path, video_caption)


# Search for music
# music_results = post_manager.search_music("Upbeat pop")
# for track in music_results:
#     print(f"{track.id}: {track.title} - {track.artist}")

# Select a track by its ID and upload a reel
# selected_track_id = music_results[0].id  # Choose the first track
# post = post_manager.upload_reel_with_music("path/to/your/reel.mp4", selected_track_id, "My fun reel!")


### Check reel with music

In [ ]:
#### This Method is not working - but needs to be reviewed through the Instagrapi community

# 5. Upload a Reel with Music:
reel_video_path = r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\VID_20151108_163255451.mp4"
reel_caption = """This is a Reel test with the same video of Otto the skateboarding bulldog.
#SkateboardingDog #GuinnessWorldRecord #Miraflores #Lima #Peru #TravelPeru #DogsofInstagram #AdventureDog #FourLeggedFun"""
location_pk = 61047499  # Malecón Cisneros - Miraflores

# Option A: Search for music on Instagram (same as before)
# search_query = "Chill vibes"
# music_results = post_manager.search_music(search_query)
# selected_track_id = music_results[0].id

# Option B: Use your own music (same as before)
music_path = r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\music_fx_emotion_driven_expectation__with_classic_mu (1).mp3"


ig_post = post_manager.upload_reel_with_music(reel_video_path, music_path, reel_caption, location_pk)

ig_posts.append(ig_post)  # Store the post object
print(ig_post)

# Create the Pandas DataFrame
df_posts = create_post_dataframe(ig_posts)
print(df_posts)

# Save the dataframe
save_post_dataframe(df_posts, filename="my_instagram_posts-3.csv")
